# Convergence
### In this notebook, we document the experiments on the reconstruction error for a fixed image resolution but three different noise models and decaying noise level. 

### We first import the packages and the data we need. For more information on the pre-saved operators and coefficients, please refer to the README.

In [ ]:
## necessary to compute reconstructions
# packages
import numpy as np
from skimage.transform import resize

# custom classes and functions
from ops import reco_op, svd_op
from filters import filter
## necessary to load pre-saved operators
import pickle

## necessary for image loading and visualization
import matplotlib.pyplot as plt
import sys

plt.rcParams.update({
    "text.usetex": True,
    "font.family": "serif",
    "text.latex.preamble":r'\usepackage{amsfonts}'
})

In [ ]:
# specify custom data location
data_folder = "spectralData"

# choose resolution
res = 32

# load pre-saved radon operator for resolution 128x128
with open(data_folder + "/radon_" + str(res) + ".obj", "rb") as f:
    r = pickle.load(f)

# load pre-saved data coefficients for the above operators and a subset of 2400 test instances of the LoDoPaB CT Dataset.
Pi = np.loadtxt(data_folder + "/pi"+ str(res) + ".txt")

# load example image and resize it to different resolutions
img = resize(plt.imread(data_folder + "/test_img.png")[..., 0], (res,res))
                
# compute and store the corresponding sinogram
sino =  r(img)

In [ ]:
U_dim = r.U.shape[-2]
V_dim = r.V.shape[-2]

full = np.max((U_dim, V_dim))

V_full = np.zeros((full,full))
V_full[:,:r.V.shape[-1]] = r.V

n = np.arange(full) + 1

problem_decay = 1/np.sqrt(np.sqrt(n))
less_decay = np.ones(full)
more_decay = 1/(n**2)

In [ ]:
# Choose number of noise samples to approximate expected error
K = 1000

# prepare noise for each sample
proto_noise = np.random.normal(scale=1, size = (V_dim,K))
# adapt proto noise for the test noise model 
noise = {'stronger': V_full@((proto_noise.T*less_decay[:V_dim]).T), 
          'problem': V_full@((proto_noise.T*problem_decay[:V_dim]).T), 
           'weaker': V_full@((proto_noise.T*more_decay[:V_dim]).T) # TODO comment on *1000 here
        }

In [ ]:
# sample a sample :)
sample = np.random.randint(K)

fig, axs = plt.subplots(1, 3, figsize=(12, 6))
axs[0].imshow(noise['stronger'][:,sample].reshape(res,-1))
axs[0].set_title(r'$\Delta_n(\nu) \propto 1$')
axs[1].imshow(noise['problem'][:,sample].reshape(res,-1))
axs[1].set_title(r'$\Delta_n(\nu) \propto 1/\sqrt{n}$')
axs[2].imshow(noise['weaker'][:,sample].reshape(res,-1))
axs[2].set_title(r'$\Delta_n(\nu) \propto 1/n^4$')

plt.show()

In [ ]:
# the noise levels we test
deltas = [0.1, 1/10**1.25, 1/10**1.5, 1/10**1.75, 0.01,1/10**2.25, 1/10**2.5, 1/10**2.75, 0.001]

# store all results for visualization in this dict:
errors = {k: {'stronger': [], 'problem': [], 'weaker':[]} for k in ['mse', 'post', 'adv']}

verbose = 1

# iterate over methods
for method in errors.keys():
    
    # iterate over noise levels
    for delta in deltas:
        
        # compute optimal filter for problem decay
        g = filter(Pi, (delta*problem_decay[:U_dim])**2, r.sigma, method = method)
        rec_op = reco_op(r.U, r.V, g, res)
        
        #reconstruct each sample and compute error for each test noise model \nu
        for nu in noise.keys():
            
            # set running error to zero
            running = 0
            
            for k in range(K):
                # the noise we add is scaled by the noise level
                reco = rec_op.reconstruct(sino+delta*noise[nu][:,k].reshape([res,-1]))
                running += np.linalg.norm(reco - img)**2
                if verbose > 1:
                    sys.stdout.write('.')

                # Save images
                if delta in [0.1,0.01,0.001] and k == 0:
                    fname = 'visualization/'+ method + '_' + nu + str(delta) + '.pdf' 
                    plt.imsave(fname, reco.reshape([res,-1]), cmap='gray')
        
            errors[method][nu].append(running/(res*res*K))
        
        if verbose > 0:
            print('Finished noise level ' + str(delta))
    if verbose > 0:        
        print('Finished method ' + method)
    


In [ ]:
g,axs= plt.subplots(1,3, sharey=True, figsize=(12,4), dpi=300)
axs[0].loglog(deltas, errors['mse']['stronger'], ls= 'solid', linewidth=2, label=r'$\Delta_n(\nu^\delta) \propto \delta^2$')
axs[0].loglog(deltas, errors['mse']['problem'], ls= 'dashed', linewidth=2, label= r'$\Delta_n(\nu^\delta) \propto \delta^2/\sqrt{n}$')
axs[0].loglog(deltas, errors['mse']['weaker'], ls= 'dotted', linewidth=2, label=r'$\Delta_n(\nu^\delta) \propto  \delta^2/n^4$')

axs[0].set_ylim(0, 0.99)
axs[0].tick_params(axis='x', labelsize=15)
axs[0].tick_params(axis='y', labelsize=15)
axs[0].set_xlabel(r'$\delta$', fontsize=20)
axs[0].set_ylabel(r'$\mathbb{E}_{\epsilon \sim \nu^{\delta} }  \left[\| R_{\mu(\delta)} (Ax+\epsilon) - x \|^2 \right ]$', fontsize=20)
axs[0].set_title(r'$R^{\mathrm{\textbf{mse}}}_{\mu(\delta)}$ with $\Delta_n(\mu(\delta)) = \delta^2/n$', fontsize=15,  pad = 10)
axs[0].invert_xaxis()
axs[0].set_aspect(1)

axs[1].loglog(deltas, errors['adv']['stronger'], ls= 'solid', linewidth=2, label=r'$\Delta_n(\nu^\delta) \propto \delta^2$')
axs[1].loglog(deltas, errors['adv']['problem'], ls= 'dashed', linewidth=2, label= r'$\Delta_n(\nu^\delta) \propto \delta^2/\sqrt{n}$')
axs[1].loglog(deltas, errors['adv']['weaker'], ls= 'dotted', linewidth=2, label=r'$\Delta_n(\nu^\delta) \propto  \delta^2/n^4$')

axs[1].tick_params(axis='x', labelsize=15)
axs[1].set_xlabel(r'$\delta$', fontsize=20)
axs[1].set_title(r'$R^{\mathrm{\textbf{adv,}}3/8}_{\mu(\delta)}$ with $\Delta_n(\mu(\delta)) = \delta^2/n$', fontsize=15, pad=10)
axs[1].invert_xaxis()
axs[1].set_aspect(1)

axs[2].loglog(deltas, errors['post']['stronger'], ls= 'solid', linewidth=2, label=r'$\Delta_n(\nu^\delta) \propto \delta^2$')
axs[2].loglog(deltas, errors['post']['problem'], ls= 'dashed', linewidth=2, label= r'$\Delta_n(\nu^\delta) \propto \delta^2/\sqrt{n}$')
axs[2].loglog(deltas, errors['post']['weaker'], ls= 'dotted', linewidth=2, label=r'$\Delta_n(\nu^\delta) \propto  \delta^2/n^4$')

axs[2].tick_params(axis='x', labelsize=15)
axs[2].set_xlabel(r'$\delta$', fontsize=20)
axs[2].legend(fontsize=15)
axs[2].set_title(r'$R^{\mathrm{\textbf{post}}}_{\mu(\delta)}$ with $\tilde{\Delta}_n(\mu(\delta)) = \delta^2/n$', fontsize=15, pad=10)
axs[2].invert_xaxis()
axs[2].set_aspect(1)